# Cylindrical Nanowire
I will use this notebook to model a cylindrical nanowire with $N_\phi$ points in $\phi$ direction and $N_z$ points along the z direction.

- BdG Hamiltonian will be used to incorporate superconductivity.

- An axial magnetic field will be assumed.

- Spin-orbit coupling and Zeeman splitting will be neglected.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import itertools
import scipy.optimize

# Define the Hamiltonian
The Hamiltonian will be a matrix of size $N_\phi N_z \times N_\phi N_z$. The order parameter and the chemical potential will be assumed to be constant in this region.

In [6]:
def calc_Hamiltonian(params):
    '''
    params is a dict with the following paramters:
    N_z   : number of points in z 
    N_phi : number of points in azimuthal direction
    t_z   : hopping paramter along z = hbar**2/(2 m a**2) where a is lattice spacing
    t_phi : hopping paramter along phi = hbar**2/(2 m a_phi**2) where a_phi is lattice spacing along 
            azimuthal direction 
    Delta : order paramter (complex number)
    mu    : chemical potential
    flux  : flux in units of h/e
    '''
    N_z = params['N_z'] 
    N_phi = params['N_phi'] 
    t_z = params['t_z']
    t_phi = params['t_phi']
    Delta = params['Delta']
    mu = params['mu']
    flux = params['flux']
    
    def matrix_ele(e1,e2):
        '''
        Returns the matrix element between points e1 and e2
        '''
        (z1,phi1) = e1
        (z2,phi2) = e2
        # onsite element
        if z1 == z2 and phi1 == phi2:
            diag_ele = 2*t_z + np.abs(t_phi)*(2 - (2*np.pi*flux/N_phi)**2) - mu
            return np.array([[diag_ele,Delta],[np.conj(Delta),-np.conj(diag_ele)]])
        # z hopping
        elif abs(z1-z2) == 1 and phi1 == phi2:
            return np.array([[-t_z,0],[0,t_z]])
        # phi hopping
        elif (phi1-phi2 == 1 or phi1-phi2 == N_phi-1)and z1 == z2:
            return np.array([[-t_phi,0],[0,np.conj(t_phi)]])
        elif (phi1-phi2 == -1 or phi1-phi2 == -N_phi+1)and z1 == z2:
            return np.conj(np.array([[-t_phi,0],[0,np.conj(t_phi)]])).T
        else:
            return np.array([[0,0],[0,0]])
        
    # the basis is given by (n_z,n_phi) where n_z = 0,..,N_z-1, n_phi = 0,...,N_phi-1
    basis = list(itertools.product(range(N_z),range(N_phi)))
    H = [matrix_ele(e1,e2) for e1 in basis for e2 in basis]
    N = N_phi*N_z

    H_ar = np.array(H,dtype=np.complex64).reshape((N,N,2,2))
    
    # magic to flatten the Hamiltonian
    # Are you wathing closely?
    H_mat = np.array([H_ar[x,:,y,:].flatten() for x in range(H_ar.shape[0]) for y in range(H_ar.shape[2])])\
    .flatten().reshape(2*N,2*N)

    return H_mat

In [8]:
# test of the Hamiltonian function
params = {
    "N_z" : 2,
    "N_phi" : 2,
    "flux" : 0e0,
    "t_z" : 5e-3,
    "Delta" : 1e-3,
    "mu" : 10e-3,
}

params["t_phi"] = 5e-3*np.exp(1j*2*np.pi*params["flux"]/params["N_phi"])

H = calc_Hamiltonian(params)
print(H)

[[ 0.020+0.j  0.001+0.j -0.010+0.j  0.000-0.j -0.005+0.j  0.000+0.j
   0.000+0.j  0.000+0.j]
 [ 0.001+0.j -0.020+0.j  0.000-0.j  0.010+0.j  0.000+0.j  0.005+0.j
   0.000+0.j  0.000+0.j]
 [-0.010-0.j  0.000+0.j  0.020+0.j  0.001+0.j  0.000+0.j  0.000+0.j
  -0.005+0.j  0.000+0.j]
 [ 0.000+0.j  0.010-0.j  0.001+0.j -0.020+0.j  0.000+0.j  0.000+0.j
   0.000+0.j  0.005+0.j]
 [-0.005+0.j  0.000+0.j  0.000+0.j  0.000+0.j  0.020+0.j  0.001+0.j
  -0.010+0.j  0.000-0.j]
 [ 0.000+0.j  0.005+0.j  0.000+0.j  0.000+0.j  0.001+0.j -0.020+0.j
   0.000-0.j  0.010+0.j]
 [ 0.000+0.j  0.000+0.j -0.005+0.j  0.000+0.j -0.010-0.j  0.000+0.j
   0.020+0.j  0.001+0.j]
 [ 0.000+0.j  0.000+0.j  0.000+0.j  0.005+0.j  0.000+0.j  0.010-0.j
   0.001+0.j -0.020+0.j]]
